# InterestBasic

Author: Sha Yuan (2018), Yuteng Huang(2025)

Installation:

```bash
pip install pandas numpy scikit-learn gensim keras nltk tensorflow
```

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Preprocessing
# Reads the file and stores strings (which is separated by '\n') into a list
# Filters the empty string which is the empty rows in train.txt
with open("data/training.txt", "r", encoding="utf-8") as f:
    train = f.read().split("\n")
    train = filter(lambda s: s, train)
    train = list(train)
    print(f"First lines:\n{train[:5]}")

First lines:
['Daniel J. Paulish', 'software engineering,software architecture,software design', 'José Cristóbal Riquelme Santos', 'neural networks,machine learning,evolutionary algorithm', 'Fernando C. N. Pereira']


In [2]:
import pandas as pd

authors = []
labels1 = []
labels2 = []
labels3 = []
for idx, item in enumerate(train):
    if idx % 2 == 0:
        authors.append(item)
    else:
        label1, label2, label3 = item.split(",")
        labels1.append(label1)
        labels2.append(label2)
        labels3.append(label3)

dfTrain = pd.DataFrame(
    {
        "author": authors,
        "Interest1": labels1,
        "Interest2": labels2,
        "Interest3": labels3,
    }
)
del authors, labels1, labels2, labels3
print(f"First authors:\n{dfTrain[:5]}")

First authors:
                           author              Interest1  \
0               Daniel J. Paulish   software engineering   
1  José Cristóbal Riquelme Santos        neural networks   
2          Fernando C. N. Pereira       computer science   
3                    Juan de Lara       computer science   
4                 Stefano Lonardi  computational biology   

                     Interest2               Interest3  
0        software architecture         software design  
1             machine learning  evolutionary algorithm  
2  natural language processing      speech recognition  
3         software engineering          model checking  
4               bioinformatics             data mining  


In [3]:
test = open("data/validation.txt", "r").read().split("\n")[:-1]
dfTest = pd.DataFrame(test)
# dfTest.rename(columns = {0: 'Name'}) # why not work here?
dfTest.columns = ["author"]
# without sort = False, the 'Name' column will be put in the last column, because it is longest.
# dfTotal = pd.concat([dfTrain, dfTest], axis = 0, sort = False).fillna('')
dfTotal = pd.concat([dfTrain, dfTest], axis=0).fillna("")
dfTotal.index = dfTotal["author"]
del test
print(dfTotal[-5:])

                              author Interest1 Interest2 Interest3
author                                                            
                                                                  
Pim van den Broek  Pim van den Broek                              
                                                                  
Christof Hasse        Christof Hasse                              
                                                                  


In [4]:
# 读取papers.txt，将其转换成pandas的DataFrame格式
papers = []
with open("data/papers.txt", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if line.startswith("#index"):
            paper_info = {}
            papers.append(paper_info)
            paper_info["index"] = line[6:].strip()
            paper_info["citation"] = ""
        elif line.startswith("#@"):
            paper_info["author"] = line[2:].strip()
        elif line.startswith("#*"):
            paper_info["paper"] = line[2:].strip()
        elif line.startswith("#t"):
            paper_info["year"] = line[2:].strip()
        elif line.startswith("#c"):
            paper_info["journal"] = line[2:].strip()
        elif line.startswith("#%"):
            paper_info["citation"] = paper_info["citation"] + line[2:] + ","
        elif line == "\n" or line == "":
            continue

dfPapers = pd.DataFrame(papers)
dfPapers.index = dfPapers["index"]
dfPapers = dfPapers.dropna()
del papers
print(dfPapers[:5])

      index                                           citation  \
index                                                            
0         0                                                      
1         1  1775098\n,1648782\n,2206608\n,2857290\n,267443...   
2         2                                                      
3         3                                           96303\n,   
4         4                                                      

                                                  author  \
index                                                      
0                          Sinjini Mitra,Marios Savvides   
1      Alberto Villa,Jocelyn Chanussot,Jon Atli Bened...   
2            Tingyuan Nie,Tomoo Kisaka,Masahiko Toyonaga   
3                                Yilmaz Erdem,Ümit Totur   
4      Sid-Ahmed Boukabara,Kevin Garrett,Wanchun Chen...   

                                                   paper  year  \
index                                             

In [5]:
"""
处理机构名和文章标题，方便做tfidf
处理方法包括：删除标点符号，统一大小写
"""


def removeSpace(x):
    new_x = ""
    for i in x.split(" "):
        new_x += i
    return new_x


"""
将期刊名+期刊名作为一个单词+论文标题组成一篇文档
例如：期刊名为Computer Vision, 论文标题为convolution neural networks
则生成文档为: Computer Vision convolution neural networks Computer_Vision
"""
# pd.set_option('display.max_rows',500)
# pd.set_option('display.max_columns',500)
# pd.set_option('display.width',1000)

dfPapers["jour_"] = dfPapers["journal"].apply(removeSpace)
dfPapers["jour_paper"] = (
    dfPapers["journal"] + " " + dfPapers["paper"] + dfPapers["jour_"]
)
jour_list = list(dfPapers["jour_"].unique())
dfPapers.drop("jour_", axis=1, inplace=True)
print(dfPapers.loc[["0", "1", "2"], ["jour_paper"]])
print(list(dfPapers.loc["1", ["jour_paper"]]))

                                              jour_paper
index                                                   
0      ICASSP (2) Improved Human Face Identification ...
1      Pattern Recognition Unsupervised methods for t...
2      ISCAS (6) A post layout watermarking method fo...
['Pattern Recognition Unsupervised methods for the classification of hyperspectral images with low spatial resolution.PatternRecognition']


In [6]:
def normalize_text(text):
    norm_text = text.lower()
    norm_text = norm_text.replace("<br />", " ")
    for char in [".", '"', ",", "(", ")", "!", "?", ":", ";", "-"]:
        norm_text = norm_text.replace(char, " " + char + " ")
    return norm_text


dfPapers["jour_paper"] = dfPapers["jour_paper"].apply(normalize_text)
docs = list(dfPapers["jour_paper"].values)
print(docs[:10])

['icassp  ( 2 )  improved human face identification using frequency domain representation of facial asymmetry . icassp ( 2 ) ', 'pattern recognition unsupervised methods for the classification of hyperspectral images with low spatial resolution . patternrecognition', 'iscas  ( 6 )  a post layout watermarking method for ip protection . iscas ( 6 ) ', 'computers & mathematics with applications some tauberian theorems for the product method of borel and cesàro summability . computers&mathematicswithapplications', 'ieee t .  geoscience and remote sensing mirs :  an all - weather 1dvar satellite data assimilation and retrieval system . ieeet . geoscienceandremotesensing', 'journal of approximation theory convergence rates of vector cascade algorithms in lp . journalofapproximationtheory', 'i .  j .  medical informatics desirability and expectations of the uk ms register :  views of people with ms . i . j . medicalinformatics', 'icisc fractional windows revisited :  improved signed - digit r

In [7]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r"\w+")
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()
    docs[idx] = tokenizer.tokenize(docs[idx])
print(docs[:50])

[['icassp', '2', 'improved', 'human', 'face', 'identification', 'using', 'frequency', 'domain', 'representation', 'of', 'facial', 'asymmetry', 'icassp', '2'], ['pattern', 'recognition', 'unsupervised', 'methods', 'for', 'the', 'classification', 'of', 'hyperspectral', 'images', 'with', 'low', 'spatial', 'resolution', 'patternrecognition'], ['iscas', '6', 'a', 'post', 'layout', 'watermarking', 'method', 'for', 'ip', 'protection', 'iscas', '6'], ['computers', 'mathematics', 'with', 'applications', 'some', 'tauberian', 'theorems', 'for', 'the', 'product', 'method', 'of', 'borel', 'and', 'cesàro', 'summability', 'computers', 'mathematicswithapplications'], ['ieee', 't', 'geoscience', 'and', 'remote', 'sensing', 'mirs', 'an', 'all', 'weather', '1dvar', 'satellite', 'data', 'assimilation', 'and', 'retrieval', 'system', 'ieeet', 'geoscienceandremotesensing'], ['journal', 'of', 'approximation', 'theory', 'convergence', 'rates', 'of', 'vector', 'cascade', 'algorithms', 'in', 'lp', 'journalofappr

In [8]:
# Remove numbers
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

print(docs[:50])

[['icassp', 'improved', 'human', 'face', 'identification', 'using', 'frequency', 'domain', 'representation', 'of', 'facial', 'asymmetry', 'icassp'], ['pattern', 'recognition', 'unsupervised', 'methods', 'for', 'the', 'classification', 'of', 'hyperspectral', 'images', 'with', 'low', 'spatial', 'resolution', 'patternrecognition'], ['iscas', 'post', 'layout', 'watermarking', 'method', 'for', 'ip', 'protection', 'iscas'], ['computers', 'mathematics', 'with', 'applications', 'some', 'tauberian', 'theorems', 'for', 'the', 'product', 'method', 'of', 'borel', 'and', 'cesàro', 'summability', 'computers', 'mathematicswithapplications'], ['ieee', 'geoscience', 'and', 'remote', 'sensing', 'mirs', 'an', 'all', 'weather', '1dvar', 'satellite', 'data', 'assimilation', 'and', 'retrieval', 'system', 'ieeet', 'geoscienceandremotesensing'], ['journal', 'of', 'approximation', 'theory', 'convergence', 'rates', 'of', 'vector', 'cascade', 'algorithms', 'in', 'lp', 'journalofapproximationtheory'], ['medical',

In [9]:
# get word_list
def processDoc(docs):
    wordList = {}
    for doc in docs:
        for word in doc:
            if word not in wordList.keys():
                wordList[word] = 1
            else:
                wordList[word] += 1
    return wordList


# 删去一些可能无用的词
wordList = processDoc(docs)
word_ind = list(wordList.keys())
word_val = list(wordList.values())
wordList = pd.DataFrame(word_val)
wordList.index = word_ind
# print(wordList[:100])
# 取出现次数超过20的词加入到tfidf 的计算中
wordList = list(wordList[wordList[0] > 20].index)
print(wordList)

['icassp', 'improved', 'human', 'face', 'identification', 'using', 'frequency', 'domain', 'representation', 'of', 'facial', 'asymmetry', 'pattern', 'recognition', 'unsupervised', 'methods', 'for', 'the', 'classification', 'hyperspectral', 'images', 'with', 'low', 'spatial', 'resolution', 'patternrecognition', 'iscas', 'post', 'layout', 'watermarking', 'method', 'ip', 'protection', 'computers', 'mathematics', 'applications', 'some', 'tauberian', 'theorems', 'product', 'borel', 'and', 'cesàro', 'summability', 'mathematicswithapplications', 'ieee', 'geoscience', 'remote', 'sensing', 'an', 'all', 'weather', 'satellite', 'data', 'assimilation', 'retrieval', 'system', 'ieeet', 'geoscienceandremotesensing', 'journal', 'approximation', 'theory', 'convergence', 'rates', 'vector', 'cascade', 'algorithms', 'in', 'lp', 'journalofapproximationtheory', 'medical', 'informatics', 'desirability', 'expectations', 'uk', 'ms', 'register', 'views', 'people', 'medicalinformatics', 'icisc', 'fractional', 'wi

In [10]:
stop_word = ["and", "a", "an", "by", "of", "on", "to", "with", "for", "the", "in"]
wordList.extend(jour_list)
for word in stop_word:
    if word in wordList:
        wordList.remove(word)
wordList = set(wordList)

for index, doc in enumerate(docs):
    # if i%1000==0:
    #     print(i)
    new_doc = []
    for word in doc:
        if word in wordList:
            new_doc.append(word)
    docs[index] = " ".join(new_doc)
print(docs[:100])

['icassp improved human face identification using frequency domain representation facial asymmetry icassp', 'pattern recognition unsupervised methods classification hyperspectral images low spatial resolution patternrecognition', 'iscas post layout watermarking method ip protection iscas', 'computers mathematics applications some tauberian theorems product method borel cesàro summability computers mathematicswithapplications', 'ieee geoscience remote sensing all weather satellite data assimilation retrieval system ieeet geoscienceandremotesensing', 'journal approximation theory convergence rates vector cascade algorithms lp journalofapproximationtheory', 'medical informatics desirability expectations uk ms register views people ms medicalinformatics', 'icisc fractional windows revisited improved signed digit representations efficient exponentiation icisc', 'ieice transactions public key encryptions invariant security reductions multi user setting ieicetransactions', 'isd proposed exten

In [11]:
# 做TFIDF 转换
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(docs))
print(type(tfidf))
print(tfidf)
# print(tfidf.shape) # (3053717, 34821) 行数为文档编号，列数为单词编号

<class 'scipy.sparse._csr.csr_matrix'>
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 29636697 stored elements and shape (3053717, 34821)>
  Coords	Values
  (0, 2275)	0.414000273664811
  (0, 8869)	0.25762854122803536
  (0, 11103)	0.28019698806206095
  (0, 11115)	0.3191061082124125
  (0, 12123)	0.25814127367149753
  (0, 14178)	0.2337942417753504
  (0, 14446)	0.4866397115949391
  (0, 14771)	0.25455488697819656
  (0, 15481)	0.2608229988726876
  (0, 26554)	0.2616261104726394
  (0, 33028)	0.15000775068284916
  (1, 5230)	0.26024440648419545
  (1, 14326)	0.38834184707712055
  (1, 15323)	0.2733367297551813
  (1, 18739)	0.2715998461478296
  (1, 19755)	0.26225533926452393
  (1, 23306)	0.2681089583087657
  (1, 23310)	0.34612439440392145
  (1, 26013)	0.24162010711917567
  (1, 26675)	0.31535347304413924
  (1, 29393)	0.2956824054200263
  (1, 32898)	0.35674285913209064
  (2, 16669)	0.3355994313363045
  (2, 16799)	0.5804756090259534
  (2, 18066)	0.3688304164803696
  :	:
  (3053714, 2550

In [12]:
# 获取每个作者发表的文章
dfPapers.index = np.arange(len(dfPapers))
dfPapers["index"] = dfPapers.index
dfPapers["index"] = dfPapers.index.astype(str)

# get cited numbers
cited = []  # cited numbers
for _ in dfPapers.index:
    cited.append("")

# print(type(dfPapers.index)) # <class 'pandas.core.indexes.numeric.Int64Index'>
# print(type(dfPapers['index'])) # <class 'pandas.core.series.Series'>

# cite = dfPapers.loc[1,['citation']]
# print(type(cite)) #<class 'pandas.core.series.Series'>
# print(list(cite))

# cite = dfPapers.loc[1,'citation']
# print(type(cite)) #<class 'str'>
# print(cite)

# for index in dfPapers['index']:
# print(type(index)) #<class 'str'>

for index in dfPapers.index:
    cite = dfPapers.loc[index, "citation"]
    if cite == "":
        continue
    for i in cite.split("\n,")[:-1]:
        if int(i) < len(dfPapers):
            cited[int(i)] = cited[int(i)] + str(index) + ":"
dfPapers["cited"] = cited
# dfPapers.drop(['year'],axis = 1,inplace = True)
print(dfPapers.loc[1, "cited"])
print(type(cited[177509]))
print("a" + cited[177509])
print(dfPapers.iloc[0:5])


<class 'str'>
a
  index                                           citation  \
0     0                                                      
1     1  1775098\n,1648782\n,2206608\n,2857290\n,267443...   
2     2                                                      
3     3                                           96303\n,   
4     4                                                      

                                              author  \
0                      Sinjini Mitra,Marios Savvides   
1  Alberto Villa,Jocelyn Chanussot,Jon Atli Bened...   
2        Tingyuan Nie,Tomoo Kisaka,Masahiko Toyonaga   
3                            Yilmaz Erdem,Ümit Totur   
4  Sid-Ahmed Boukabara,Kevin Garrett,Wanchun Chen...   

                                               paper  year  \
0  Improved Human Face Identification Using Frequ...  2006   
1  Unsupervised methods for the classification of...  2013   
2  A post layout watermarking method for IP prote...  2005   
3  Some Tauberian theorem

In [13]:
dfPapers["author"] += "," * 22
for i in range(23):
    dfPapers["author" + str(i + 1)] = dfPapers["author"].apply(
        lambda x: x.split(",")[i]
    )
dfPapers["author"] = dfPapers["author"].apply(lambda x: x[:-6])
print(dfPapers[:5])

  index                                           citation  \
0     0                                                      
1     1  1775098\n,1648782\n,2206608\n,2857290\n,267443...   
2     2                                                      
3     3                                           96303\n,   
4     4                                                      

                                              author  \
0      Sinjini Mitra,Marios Savvides,,,,,,,,,,,,,,,,   
1  Alberto Villa,Jocelyn Chanussot,Jon Atli Bened...   
2  Tingyuan Nie,Tomoo Kisaka,Masahiko Toyonaga,,,...   
3            Yilmaz Erdem,Ümit Totur,,,,,,,,,,,,,,,,   
4  Sid-Ahmed Boukabara,Kevin Garrett,Wanchun Chen...   

                                               paper  year  \
0  Improved Human Face Identification Using Frequ...  2006   
1  Unsupervised methods for the classification of...  2013   
2  A post layout watermarking method for IP prote...  2005   
3  Some Tauberian theorems for the product

In [14]:
for i in range(23):
    t1 = (
        dfPapers.groupby([f"author{i+1}"])["index"]
        .apply(lambda x: ":".join(x))
        .reset_index()
    )
    t1.columns = ["author", f"paper{i+1}"]
    # print(t1)
    dfTotal = pd.merge(
        dfTotal, t1, left_index=True, right_on="author", how="left"
    ).fillna("")
    dfTotal = dfTotal.set_index("author")
del t1
print(dfTotal[:5])

                                                      author_x  \
author                                                           
Daniel J. Paulish                            Daniel J. Paulish   
José Cristóbal Riquelme Santos  José Cristóbal Riquelme Santos   
Fernando C. N. Pereira                  Fernando C. N. Pereira   
Juan de Lara                                      Juan de Lara   
Stefano Lonardi                                Stefano Lonardi   

                                            Interest1  \
author                                                  
Daniel J. Paulish                software engineering   
José Cristóbal Riquelme Santos        neural networks   
Fernando C. N. Pereira               computer science   
Juan de Lara                         computer science   
Stefano Lonardi                 computational biology   

                                                  Interest2  \
author                                                        
Daniel J. P

In [15]:
for i in range(23):
    t1 = (
        dfPapers.groupby([f"author{i+1}"])["cited"]
        .apply(lambda x: ":".join(x))
        .reset_index()
    )
    # print(t1)
    t1.columns = ["author", f"cited{i+1}"]
    dfTotal = pd.merge(dfTotal, t1, on=["author"], how="left").fillna("")
del t1
print(dfTotal[0:5])

                           author                        author_x  \
0               Daniel J. Paulish               Daniel J. Paulish   
1  José Cristóbal Riquelme Santos  José Cristóbal Riquelme Santos   
2          Fernando C. N. Pereira          Fernando C. N. Pereira   
3                    Juan de Lara                    Juan de Lara   
4                 Stefano Lonardi                 Stefano Lonardi   

               Interest1                    Interest2               Interest3  \
0   software engineering        software architecture         software design   
1        neural networks             machine learning  evolutionary algorithm   
2       computer science  natural language processing      speech recognition   
3       computer science         software engineering          model checking   
4  computational biology               bioinformatics             data mining   

                         author_y  \
0               Daniel J. Paulish   
1  José Cristóbal Riquel

In [ ]:
### VerySlow
# print(tfidf.shape) # (3053717, 34821) 行数为文档编号，列数为单词编号
print(tfidf[1][0])
tfidf_weight_array = np.zeros([len(dfTotal), tfidf.shape[1]])
# print(dfTotal.shape) # there is 11367 authors
# print(dfPapers.shape) # there is 3053717 papers
# print(tfidf_weight_array.shape) # (11367, 34821)

for i in range(23):
    # for i in range(2):
    col_name = "paper" + str(i + 1)
    # for j in [0,1]:
    for j in dfTotal.index:
        paper_list = dfTotal.loc[j, col_name].split(":")
        # print(paperlist)
        for paper_index in paper_list:
            if paper_index == "":
                continue
            tfidf_weight = tfidf[int(paper_index)][0]
            # print(type(tfidf_weight)) # <class 'scipy.sparse.csr.csr_matrix'>
            # print(tfidf_weight)
            for words in tfidf_weight.indices:
                # csr_matrix.indices表示各个数据在某行的下标，即第几列，下标从0开始
                # print(words)
                first_vec = tfidf_weight.getcol(words).data[0]
                if j < 7:
                    # csr_matrix.getcol(i),获得矩阵的第i列，返回一个列向量
                    tfidf_weight_array[j, words] = first_vec * (1 - 0.1 * i)
                else:
                    tfidf_weight_array[j, words] = first_vec * 0.3
print(tfidf_weight_array[:5, :])


In [16]:
import numpy as np

num_authors = len(dfTotal)
num_words = tfidf.shape[1]
tfidf_weight_array = np.zeros((num_authors, num_words))

# Step 1. 预构建作者的所有论文索引 + 权重
print("Pre-building paper index and weights...")
author_paper_map = [[] for _ in range(num_authors)]
author_weight_map = [[] for _ in range(num_authors)]

for i in range(23):
    col_name = f"paper{i+1}"
    weight = (1 - 0.1 * i)
    for j, paper_str in enumerate(dfTotal[col_name]):
        if not paper_str:
            continue
        papers = paper_str.split(':')
        for p in papers:
            if p == "":
                continue
            author_paper_map[j].append(int(p))
            author_weight_map[j].append(weight if j < 7 else 0.3)

# Step 2. 向量化组合
print("Computing weighted vectors...")
for j in range(num_authors):
    if not author_paper_map[j]:
        continue
    print(f"Processing author {j+1}/{num_authors}...")
    papers_idx = author_paper_map[j]
    weights = np.array(author_weight_map[j])
    tfidf_subset = tfidf[papers_idx]  # 稀疏矩阵切片
    weighted_sum = tfidf_subset.multiply(weights[:, None]).sum(axis=0)
    tfidf_weight_array[j, :] = np.array(weighted_sum)
print(tfidf_weight_array[:5, :])


Pre-building paper index and weights...


MemoryError: 

In [ ]:
print(type(tfidf_weight_array))
print(tfidf_weight_array[0, :])

for i in range(7):
    col_name = "cited" + str(i + 1)
    for j in dfTotal.index:
        paper_list = dfTotal.loc[j, col_name].split(":")
        for paper_index in paper_list:
            if paper_index == "":
                continue
            tfidf_weight = tfidf[int(paper_index)][0]
            for words in tfidf_weight.indices:
                # print(words)
                tfidf_weight_array[j, words] = (
                    tfidf_weight.getcol(words).data[0] * (1 - i * 0.1) * 0.1
                )
print(tfidf_weight_array[0:5, :])

<class 'numpy.ndarray'>
[ 0.  0.  0. ...,  0.  0.  0.]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [ ]:
'''
用keras 的普通的DNN训练
共700+种class
label里出现的class 按出现顺序分别给 0.95 0.85 0.75 的分数，其他给0.1
'''

labels = pd.read_csv('/Users/yuansha/Desktop/portrait/data of task2/labels.txt',header = None)
y_total = 0.1 * np.ones([len(dfTotal),len(labels)],dtype='float64')
print(y_total.shape)

labels['idx'] = labels.index
labels.index = labels[0]
labels.drop(0,axis = 1,inplace = True)

#print(dfTrain.iloc[1])
#print(dfTrain.loc[1,'interest1'])

for idx in dfTrain.index:
    for j in range(3):
        y_total[idx,labels.loc[dfTrain.loc[idx,'Interest'+str(j+1)],'idx']] = 0.95-0.1*j
print(y_total[0:5])


(11367, 789)
[[ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]]


In [ ]:
idtolabels = labels.copy()
print(type(idtolabels))
idtolabels['name'] = idtolabels.index
idtolabels.index = idtolabels.idx
idtolabels.drop('idx',inplace = True,axis = 1)
print(idtolabels.iloc[0:5])

<class 'pandas.core.frame.DataFrame'>
                        name
idx                         
0    abstract interpretation
1             access control
2             access network
3            active learning
4       activity recognition


In [ ]:
class Task2_Model():

        def __init__(self,output_dim):
            '''
            model1: tfidf
            '''
            self.model = Sequential()
            self.model.add(Dense(1000,activation='relu',input_dim=tfidf_weight_array.shape[1]))
            # print(tfidf.shape) # (3053717, 34821) 行数为文档编号，列数为单词编号
            self.model.add(Dense(output_dim,activation='sigmoid'))

            self.model.compile(loss='categorical_crossentropy',
                                optimizer='adadelta',
                                metrics=['acc'])

        def train_model(self,x_train,y_train,sp,epochs):
            if sp < 1.0:
                self.model.fit(x_train[0:int(len(x_train)*sp), :],y_train[0:int(len(x_train)*sp),:],
                                batch_size=30,
                                epochs=epochs,
                                validation_data=(x_train[int(len(x_train)*sp):,:],y_train[int(len(y_train)*sp):,:]))

            if sp == 1.0:
                self.model.fit(x_train,y_train,
                            batch_size=30,
                            epochs=epochs,
                            validation_data=(x_train,y_train))
            history = self.model.fit.history

        def save_model(self,path):
            self.model.save(path + '_1_lda.h5')

        def load_model(slef, path):
            self.model.load_weights(path +'_1_lda.h5')

        def predict(self,fea1):
            submission = list()
            preds = self.model.predict(fea1)
            for idx in range(len(fea1)):
                a = np.argsort(preds[idx])[-5:] # np.argsort升序排列并返回下标
                a_label = idtolabels.ix[a].values
                # print(a_label)
                submission.append(a_label)
            return submission


        def getScore(self,X,y,idtolabel):
            preds = self.model.predict(X)
            score = 0
            for idx in range(len(X)):
                a = np.argsort(preds[idx])[-3:]
                a_label = idtolabel.ix[a].values
                # print(a_label)
                for j in range(3):
                    for k in range(3):
                        if a_label[j]==y[idx,k]:
                            score+=1
                            break
            return score/len(X)/3
        def summary(self):
            print(self.model.summary())


In [ ]:
model = Task2_Model(len(labels))
hist = model.train_model(tfidf_weight_array[:6000],y_total[:6000],1,13)
    # print(model.getScore(tfidf_weight_array[4800:6000],dfTrain.ix[4800:6000,1:].values,idtolabels))
submission = model.predict(tfidf_weight_array[6000:])


Train on 6000 samples, validate on 6000 samples
Epoch 1/13
6000/6000 [==============================] - 142s 24ms/step - loss: 541.1954 - acc: 0.0447 - val_loss: 541.1297 - val_acc: 0.0460
Epoch 2/13
6000/6000 [==============================] - 132s 22ms/step - loss: 541.0887 - acc: 0.0602 - val_loss: 541.0165 - val_acc: 0.1037
Epoch 3/13
6000/6000 [==============================] - 132s 22ms/step - loss: 540.9739 - acc: 0.1405 - val_loss: 540.8957 - val_acc: 0.1712
Epoch 4/13
6000/6000 [==============================] - 132s 22ms/step - loss: 540.8468 - acc: 0.1938 - val_loss: 540.7431 - val_acc: 0.1945
Epoch 5/13
6000/6000 [==============================] - 136s 23ms/step - loss: 540.6921 - acc: 0.2430 - val_loss: 540.5517 - val_acc: 0.3028
Epoch 6/13
6000/6000 [==============================] - 133s 22ms/step - loss: 540.5174 - acc: 0.2962 - val_loss: 540.3611 - val_acc: 0.3525
Epoch 7/13
6000/6000 [==============================] - 138s 23ms/step - loss: 540.3341 - acc: 0.3597 - va

/Users/yuansha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [ ]:
# the function plot loss and acc
def training_vis(hist):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    acc = hist.history['acc']
    val_acc = hist.history['val_acc']

    # make a figure
    fig = plt.figure(figsize=(8,4))
    # subplot loss
    ax1 = fig.add_subplot(121)
    ax1.plot(loss,label='train_loss')
    ax1.plot(val_loss,label='val_loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss on Training and Validation Data')
    ax1.legend()
    # subplot acc
    ax2 = fig.add_subplot(122)
    ax2.plot(acc,label='train_acc')
    ax2.plot(val_acc,label='val_acc')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Accuracy  on Training and Validation Data')
    ax2.legend()
    plt.tight_layout()

In [ ]:
training_vis(hist)

AttributeError: 'NoneType' object has no attribute 'history'

In [ ]:
print(hist.history)

AttributeError: 'NoneType' object has no attribute 'history'

In [ ]:
with open('validation2.csv','w') as f:
    f.write('<task2>\nname')
    for i in range(5):
        f.write('\tinterest'+str(i+1))
    f.write('\n')
    for i in range(len(submission)):
        f.write('%s'%(dfValidation.ix[i,0]))
        for j in range(5):
            f.write('\t%s'%(submission[i][j][0]))
        f.write('\n')
    f.write('</task2>')